Скрипт позволяет добавлять в экспериментальные "виртуальные" маркеры, координаты которых есть среднее арифметическое координат некоторых других маркеров.

In [ ]:
import numpy as np
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
from numpy import linalg as LA

In [5]:
#Все маркеры
markers = ['RAH', 'RPH', 'LAH', 'LPH', 'C7', 'T8', 'IJ', 'PX','RSC', 'RAC','LSC', 'LAC', 'RAI', 'RPC',
           'RTS', 'RAA', 'LAI', 'LTS', 'LAA', 'LPC', 'REM', 'REL', 'LEM', 'LEL', 'RRS', 'RUS', 'LRS', 'LUS', 'LSJ', 'RSJ']
#Маркеры, условно неподвижные относительно тела (непосредственно связаны со скелетом)
stablemarkers = ['T8', 'IJ', 'PX', 'C7']
#Маркеры лопатки
markersscap = ['RTS', 'RAI', 'RAA']
#Хранит названия всех колонок в списке
cols = []
#Хранит названия колонок неподвижных маркеров в списке
stablecols = []
#Хранит названия колонок маркеров лопатки в списке
scapulacols = []
#Data from original file
datalines = []
#Хранит названия колонок в строке
csvcols = ''
#Формирование списков
for i in markers:
    cols.append(i + 'x')
    cols.append(i + 'y')
    cols.append(i + 'z')
    csvcols += cols[-3] + '\t' + cols[-2] + '\t' + cols[-1] + '\t'
for i in stablemarkers:
    stablecols.append(i + 'x')
    stablecols.append(i + 'y')
    stablecols.append(i + 'z')
for i in markersscap:
    scapulacols.append(i + 'x')
    scapulacols.append(i + 'y')
    scapulacols.append(i + 'z')

In [7]:
#Здесь указывается анализируемый файл
file = 'smoothed_trc\\very long static 14_shifted_smoothed.trc'
with open (file, 'r') as fp:
    datalines = fp.readlines()
    #Удаляется ненужная техническая информация
    infolines = datalines[0:5]
    datalines = datalines[5:-1]
textdatalines = ''
textinfodatalines = ''
for i in datalines:
    textdatalines += i + '\n'
#df хранит данные оригинального файла
df = pd.read_csv(StringIO(textdatalines), sep = '\t', names = ['Frame#', 'Time'] + cols)

In [8]:
#Словарь, сопоставляющий группе маркеров для усреднения конкретный набор этих маркеров
#Можно своему ключу сопоставить некоторую группу маркеров. В итоге в экспериментальные данные будет добавлен
#новый виртуальный маркер
new_mean_markers = {}
new_mean_markers['RLPH'] = ['RPH', 'LPH'] #Center of dorsal head
new_mean_markers['RLSC'] = ['RSC', 'LSC'] #Center of sternum's top
new_mean_markers['RLTS'] = ['RTS', 'LTS'] #Markers between scapulas
new_mean_markers['RLAI'] = ['RAI', 'LAI'] #Markers between scapulas
new_mean_markers['RELEM'] = ['REL', 'REM'] #Humerus and ulna centrums
new_mean_markers['RRSUS'] = ['RRS', 'RUS'] 
new_mean_markers['LELEM'] = ['LEL', 'LEM']
new_mean_markers['LRSUS'] = ['LRS', 'LUS']
new_mean_markers['RSCAP'] = ['RAA', 'RAI', 'RTS'] #scapular contact
new_mean_markers['LSCAP'] = ['LAA', 'LAI', 'LTS']

In [9]:
#Чтобы не портить исходные данные, используем df1
df1 = df.copy()
#Добавляются новые маркеры, являющиеся ключами в словаре new_mean_markers
for i in new_mean_markers:
    for j in ['x', 'y', 'z']:
        data = np.zeros(len(df1.index))
        for k in new_mean_markers[i]:
            data += df1[k + j]
        data /= len(new_mean_markers[i])
        df1.insert(len(df1.columns), i + j, data, False)

In [11]:
#30 - исходное число маркеров, nummarkers - итоговое число маркеров
nummarkers = 30 + len(new_mean_markers)
#Запись технической инофрмации в trc файл
newinfolines = infolines.copy()
#В конце указывается имя для нового файла с координатами
newinfolines[0] = "PathFileType\t4\t(X/Y/Z)\\" + file[0:-4] + '_adv_mark.trc\n'
newinfolines[2] = '100\t100\t' + str(len(df1)+1) + '\t' +str(nummarkers)+'\tmm\t100\t' + str(df1['Time'].values[0]) + '\t' + str(len(df1)+1) + '\n'
newinfolines[3] = 'Frame#\tTime'
#Запись кординат
for i in markers:
    newinfolines[3] += '\t' + i
for i in new_mean_markers:
    newinfolines[3] += '\t' + i
newinfolines[3] += '\n'
newinfolines[4] = '\t\t'
for i in range(1, nummarkers+1):
    newinfolines[4] += 'X' + str(i) + '\t'
    newinfolines[4] += 'Y' + str(i) + '\t'
    newinfolines[4] += 'Z' + str(i) + '\t'
newinfolines[4] += '\n'
#Запись результата в файл
newtextlines = df1.to_csv(sep = '\t', header=False, index = False)
newtextlines = ''.join(newinfolines) + newtextlines
with open (file[0:-4] + '_adv_mark.trc', 'w+') as fp:
    fp.write(newtextlines)

Frame#	Time	RAH	RPH	LAH	LPH	C7	T8	IJ	PX	RSC	RAC	LSC	LAC	RAI	RPC	RTS	RAA	LAI	LTS	LAA	LPC	REM	REL	LEM	LEL	RRS	RUS	LRS	LUS	LSJ	RSJ	RLPH	RLSC	RLTS	RLAI	RELEM	RRSUS	LELEM	LRSUS	RSCAP	LSCAP

